# **PCY Algorithm (PySpark)**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_set
from itertools import combinations
from collections import defaultdict

In [2]:
# PCY Algorithm Class
class PCYAlgorithm:
    def __init__(self, support_threshold, confidence_threshold, num_buckets=100):
        """
        Initialize the PCY Algorithm with user-defined thresholds.

        :param support_threshold: Minimum support count to consider an item/pair frequent.
        :param confidence_threshold: Minimum confidence level for association rules.
        :param num_buckets: Number of hash buckets for pair counting.
        """
        self.support_threshold = support_threshold
        self.confidence_threshold = confidence_threshold
        self.num_buckets = num_buckets
        self.bucket_counts = defaultdict(int)  # Dictionary to store bucket counts

    def hash_pair(self, pair):
        """
        Hash function to map item pairs into buckets.

        :param pair: A tuple representing an item pair (item1, item2).
        :return: A hash bucket index.
        """
        return hash(pair) % self.num_buckets

    def count_frequent_items(self, baskets):
        """
        First pass of the PCY algorithm: Count individual items.

        :param baskets: List of baskets (transactions), where each basket is a set of items.
        :return: Dictionary of frequent items sorted in descending order of count.
        """
        item_counts = defaultdict(int)

        # Count occurrences of each item in all baskets
        for basket in baskets:
            for item in basket:
                item_counts[item] += 1

        # Sort items by frequency in descending order and return
        return dict(sorted(item_counts.items(), key=lambda x: x[1], reverse=True))

    def count_frequent_pairs(self, baskets, frequent_items):
        """
        Second pass of the PCY algorithm: Count item pairs.

        :param baskets: List of baskets (transactions).
        :param frequent_items: Dictionary of frequent items from the first pass.
        :return: Dictionary of frequent pairs sorted in descending order of count.
        """
        pair_counts = defaultdict(int)

        # Iterate through each basket
        for basket in baskets:
            # Keep only frequent items in the basket
            valid_items = [item for item in basket if item in frequent_items]

            # Generate all possible item pairs
            for pair in combinations(valid_items, 2):
                pair_counts[pair] += 1  # Increment pair count
                self.bucket_counts[self.hash_pair(pair)] += 1  # Hash pair into a bucket

        # Sort pairs by frequency in descending order and return
        return dict(sorted(pair_counts.items(), key=lambda x: x[1], reverse=True))

    def generate_association_rules(self, frequent_pairs, frequent_items):
        """
        Generate association rules based on confidence.

        :param frequent_pairs: Dictionary of frequent item pairs.
        :param frequent_items: Dictionary of frequent single items.
        :return: List of association rules (item1 -> item2) with confidence.
        """
        rules = []

        # Compute confidence for each frequent pair
        for (item1, item2), count in frequent_pairs.items():
            confidence1 = count / frequent_items[item1]  # P(Item2 | Item1)
            confidence2 = count / frequent_items[item2]  # P(Item1 | Item2)

            # Add rules if confidence meets the threshold
            if confidence1 >= self.confidence_threshold:
                rules.append((item1, item2, confidence1))
            if confidence2 >= self.confidence_threshold:
                rules.append((item2, item1, confidence2))

        # Sort rules by confidence in descending order and return
        return sorted(rules, key=lambda x: x[2], reverse=True)

In [3]:
# Function to load and preprocess data from CSV
def load_data():
    """
    Load transaction data from baskets.csv and format it into a list of baskets.

    :return: List of baskets (each basket is a set of items).
    """
    # Start a Spark session
    spark = SparkSession.builder.appName("PCYAlgorithm").getOrCreate()

    # Read the CSV file
    df = spark.read.csv("baskets.csv", header=True, inferSchema=True)

    # Group data by Member_number and Date, aggregating items into a list
    baskets_df = df.groupBy("Member_number", "Date").agg(collect_set("itemDescription").alias("basket"))

    # Convert Spark DataFrame to a list of baskets
    baskets = [row["basket"] for row in baskets_df.collect()]

    # Stop Spark session
    spark.stop()

    return baskets

# Function to execute the PCY algorithm
def run_pcy_algorithm(baskets, support_threshold, confidence_threshold):
    """
    Run the PCY Algorithm on the given transaction baskets.

    :param baskets: List of baskets (transactions).
    :param support_threshold: Minimum support threshold for frequent items and pairs.
    :param confidence_threshold: Minimum confidence threshold for association rules.
    :return: Frequent items, frequent pairs, and association rules.
    """
    # Create an instance of the PCYAlgorithm
    pcy = PCYAlgorithm(support_threshold, confidence_threshold)

    # First pass: Count frequent individual items
    frequent_items = pcy.count_frequent_items(baskets)

    # Second pass: Count frequent item pairs
    frequent_pairs = pcy.count_frequent_pairs(baskets, frequent_items)

    # Generate association rules based on confidence
    association_rules = pcy.generate_association_rules(frequent_pairs, frequent_items)

    return frequent_items, frequent_pairs, association_rules

# Function to display results
def display_results(frequent_items, frequent_pairs, association_rules):
    """
    Display the top 30 frequent items, frequent pairs, and association rules in descending order.
    """
    print("\n=== Top 30 Frequent Items ===")
    for item, count in sorted(frequent_items.items(), key=lambda x: x[1], reverse=True)[:30]:
        print(f"Item: {item}, Count: {count}")

    print("\n=== Top 30 Frequent Pairs ===")
    for pair, count in sorted(frequent_pairs.items(), key=lambda x: x[1], reverse=True)[:30]:
        print(f"Pair: {pair}, Count: {count}")

    print("\n=== Top 30 Association Rules (Sorted by Confidence) ===")
    for item1, item2, confidence in sorted(association_rules, key=lambda x: x[2], reverse=True)[:30]:
        print(f"{item1} -> {item2} (Confidence: {confidence:.2f})")

In [4]:
# Main Execution
if __name__ == "__main__":
    # Step 1: Load transaction data
    baskets = load_data()

    # Step 2: Set thresholds and run the PCY Algorithm
    support_threshold = 2  # Minimum count for items to be considered frequent
    confidence_threshold = 0.5  # Minimum confidence for association rules
    frequent_items, frequent_pairs, association_rules = run_pcy_algorithm(baskets, support_threshold, confidence_threshold)

    # Step 3: Display results
    display_results(frequent_items, frequent_pairs, association_rules)


=== Top 30 Frequent Items ===
Item: whole milk, Count: 2363
Item: other vegetables, Count: 1827
Item: rolls/buns, Count: 1646
Item: soda, Count: 1453
Item: yogurt, Count: 1285
Item: root vegetables, Count: 1041
Item: tropical fruit, Count: 1014
Item: bottled water, Count: 908
Item: sausage, Count: 903
Item: citrus fruit, Count: 795
Item: pastry, Count: 774
Item: pip fruit, Count: 734
Item: shopping bags, Count: 712
Item: canned beer, Count: 702
Item: bottled beer, Count: 678
Item: whipped/sour cream, Count: 654
Item: newspapers, Count: 582
Item: frankfurter, Count: 565
Item: brown bread, Count: 563
Item: domestic eggs, Count: 555
Item: pork, Count: 555
Item: butter, Count: 527
Item: fruit/vegetable juice, Count: 509
Item: beef, Count: 508
Item: curd, Count: 504
Item: margarine, Count: 482
Item: coffee, Count: 473
Item: frozen vegetables, Count: 419
Item: chicken, Count: 417
Item: white bread, Count: 359

=== Top 30 Frequent Pairs ===
Pair: ('whole milk', 'other vegetables'), Count: 22